In [61]:
import pandas as pd
import numpy as np
from datetime import datetime

#Statistical LTV
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import calibration_and_holdout_data, summary_data_from_transaction_data

#ML Approach to LTV
import lightgbm as lgb

#Evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#### **Data Pre-Processing**

In [2]:
retaildf = pd.read_csv('data/CLV_online_retail.csv', engine='python')

In [3]:
retaildf

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/11 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/11 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/11 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/11 12:50,4.15,12680.0,France


In [4]:
retaildf['InvoiceDate'] = pd.to_datetime(retaildf.InvoiceDate, format = '%m/%d/%y %H:%M')

In [5]:
#separate out the various parts of the date
retaildf['date'] = pd.to_datetime(retaildf.InvoiceDate.dt.date)
retaildf['time'] = retaildf.InvoiceDate.dt.time
retaildf['hour'] = retaildf['time'].apply(lambda x: x.hour)
retaildf['weekend'] = retaildf['date'].apply(lambda x: x.weekday() in [5, 6])
retaildf['dayofweek'] = retaildf['date'].apply(lambda x: x.dayofweek)

In [6]:
retaildf.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
date                0
time                0
hour                0
weekend             0
dayofweek           0
dtype: int64

In [7]:
retaildf.dropna(inplace=True)

In [13]:
retaildf = retaildf[~retaildf.Quantity < 0]
retaildf.shape

(397924, 13)

In [14]:
retaildf.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,time,hour,weekend,dayofweek
49122,540502,21181,PLEASE ONE PERSON METAL SIGN,48,2011-01-09 10:50:00,1.85,13183.0,United Kingdom,2011-01-09,10:50:00,10,True,6
231993,557296,23322,LARGE WHITE HEART OF WICKER,2,2011-06-19 13:51:00,2.95,15334.0,United Kingdom,2011-06-19,13:51:00,13,True,6
369395,569027,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-09-30 10:19:00,4.15,13089.0,United Kingdom,2011-09-30,10:19:00,10,False,4
190405,553191,85214,TUB 24 PINK FLOWER PEGS,1,2011-05-15 13:22:00,1.65,13118.0,United Kingdom,2011-05-15,13:22:00,13,True,6
117355,546383,85099F,JUMBO BAG STRAWBERRY,10,2011-03-11 12:54:00,1.95,13654.0,United Kingdom,2011-03-11,12:54:00,12,False,4


In [15]:
salesdf = retaildf.groupby('date')['Quantity'].sum()
fig = px.line(salesdf, title='Sales')
fig.update_layout(
    xaxis_title = 'Date',
    yaxis_title = 'Quantity'
)
fig.show()
print(retaildf['date'].max() - retaildf['date'].min())

373 days 00:00:00


##### From this plot, we see that we have about 1 year of data. For our ML approach (LightGBM model), the train/test splits will be the following segments:
##### &emsp;1. Training Features Period: 2011-01-01 to 2011-06-11
##### &emsp;2. Training Target Period: 2011-06-12 to 2011-09-09
##### &emsp;3. Testing Features Period: 2011-04-02 to 2011-09-10
##### &emsp;4. Testing Target Period: 2011-09-11 to 2011-12-09

##### The choice of time periods was arbitrary. My goal was to have my target period be roughly half the duration of my training period for the features and target.

In [16]:
#Dataset info
print(f'Total Number of Purchases: {retaildf.shape[0]}')
print(f'Total Number of transactions: {retaildf.InvoiceNo.nunique()}')
print(f'Total Unique Days: {retaildf.date.nunique()}')
print(f'Total Unique Customers: {retaildf.CustomerID.nunique()}')
print(f"We are predicting {(retaildf['date'].max() - datetime(2011, 9, 11)).days} days")

Total Number of Purchases: 397924
Total Number of transactions: 18536
Total Unique Days: 305
Total Unique Customers: 4339
We are predicting 89 days


#### **Baseline model: BG/NBD and Gamma-Gamma models for CLV**
##### Check out the **BGNBD_analysis.ipynb** file in this repository for a more in-depth walkthrough of the implementation of these models for predicting CLV. In this experiment, I am just going to fit and evaluate the models and focus more on the ML technique.

##### **Data Prep**
##### This modeling technique requires data on the transaction level, and the features need to be in RFM format (must have *recency*, *frequency*, *monetary_value*, and *T*(age) columns). For this data, I will calculate revenue as the product of quantity and item price.

In [17]:
#Get revenue column
retaildf['Revenue'] = retaildf['Quantity'] * retaildf['UnitPrice']

#Context data for the revenue (date & customerID)
id_lookup = retaildf[['CustomerID', 'InvoiceNo', 'date']].drop_duplicates()
id_lookup.index = id_lookup['InvoiceNo']
id_lookup = id_lookup.drop('InvoiceNo', axis=1)

transactions_data = pd.DataFrame(retaildf.groupby('InvoiceNo')['Revenue'].sum()).join(id_lookup)

In [18]:
transactions_data.head()

,Revenue,CustomerID,date
InvoiceNo,,,
536365,139.12,17850.0,2010-12-01
536366,22.20,17850.0,2010-12-01
536367,278.73,13047.0,2010-12-01
536368,70.05,13047.0,2010-12-01
536369,17.85,13047.0,2010-12-01


In [19]:
#Spit into train - test
rfm_train_test = calibration_and_holdout_data(transactions_data, 'CustomerID', 'date',
                                        calibration_period_end='2011-09-10',
                                        monetary_value_col = 'Revenue')   

#Selecting only customers with positive value in the calibration period (otherwise Gamma-Gamma model doesn't work)
rfm_train_test = rfm_train_test.loc[rfm_train_test['monetary_value_cal'] > 0, :]

In [20]:
print(rfm_train_test.shape)
rfm_train_test.head()

(1956, 7)


,frequency_cal,recency_cal,T_cal,monetary_value_cal,frequency_holdout,monetary_value_holdout,duration_holdout
CustomerID,,,,,,,
12347.0,4.0,238.0,277.0,519.7675,2.0,759.570000,90.0
12348.0,2.0,110.0,268.0,297.2200,1.0,310.000000,90.0
12352.0,3.0,34.0,206.0,421.7700,3.0,314.743333,90.0
12356.0,1.0,80.0,235.0,481.4600,1.0,58.350000,90.0
12359.0,2.0,142.0,241.0,1474.1150,1.0,2876.850000,90.0


##### **Tuning and Training**
##### We need to make sure that montary value and frequency are not highly correlated to satisfy a basic assumption of this model.

In [21]:
#Train Gamma-Gamma
print(rfm_train_test[['monetary_value_cal', 'frequency_cal']].corr())

                    monetary_value_cal  frequency_cal
monetary_value_cal            1.000000       0.105329
frequency_cal                 0.105329       1.000000


##### We see that there is a low correlation, so we can continue.

##### To determine the optimal L2 penalty coefficient, we will perform grid search

In [98]:
# Tune BG/NBD model

l2_coefs = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
l2_arr = []
rmse_arr = []
for coef in l2_coefs:
    # Fit model for each L2 coefficient
    model = BetaGeoFitter(penalizer_coef=coef)
    model.fit(rfm_train_test['frequency_cal'], 
        rfm_train_test['recency_cal'], 
        rfm_train_test['T_cal'])
    # Predicting the frequency for the holdout period for all customers. 
    pred_freqs = pd.DataFrame(model.predict(89,
                        rfm_train_test['frequency_cal'], 
                        rfm_train_test['recency_cal'], 
                        rfm_train_test['T_cal']), columns=['pred_frequency']).reset_index().fillna(0)
    # Merging the two dataframes and dropping NaN values. 
    new_df = rfm_train_test.reset_index().merge(pred_freqs, on='CustomerID').dropna()

    # Calcuclate the mae
    rmse_score = np.sqrt(mean_squared_error(new_df['frequency_holdout'],new_df['pred_frequency']))
    l2_arr.append(coef)
    rmse_arr.append(rmse_score)

# Save results in df
resl = pd.DataFrame(np.array(rmse_arr), columns=['rmse_score'])\
             .merge(pd.DataFrame(np.array(l2_arr), columns=['L2 coefs']), right_index=True, left_index=True)
resl

,rmse_score,L2 coefs
0,1.906153,0.0
1,1.791718,0.1
2,1.772594,0.2
3,1.762634,0.3
4,1.756178,0.4
5,1.751533,0.5
6,1.747976,0.6
7,1.745138,0.7
8,1.742806,0.8
9,1.740848,0.9


In [86]:
#Train the BG/NBD model
bgf = BetaGeoFitter(penalizer_coef=0.1)
bgf.fit(rfm_train_test['frequency_cal'], rfm_train_test['recency_cal'], rfm_train_test['T_cal'])


<lifetimes.BetaGeoFitter: fitted with 1956 subjects, a: 0.00, alpha: 23.81, b: 0.01, r: 0.45>

In [97]:
# Tune Gamma-Gamma model

l2_coefs = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
l2_arr = []
mse_arr = []
for coef in l2_coefs:
    # Fit model for each L2 coefficient
    model = GammaGammaFitter(penalizer_coef=coef)
    model.fit(rfm_train_test['frequency_cal'], 
        rfm_train_test['monetary_value_cal'])
    # Predictingthe monetary value for all customers. 
    pred_freqs = pd.DataFrame(model.conditional_expected_average_profit( 
                                 rfm_train_test['frequency_cal'], rfm_train_test['monetary_value_cal']), columns=['pred_monetary_value']).reset_index().fillna(0)
    # Merging the two dataframes and dropping NaN values. 
    new_df = rfm_train_test.reset_index().merge(pred_freqs, on='CustomerID').dropna()

    # Calcuclate the mae
    mae_score = np.sqrt(mean_squared_error(new_df['monetary_value_holdout'],new_df['pred_monetary_value']))
    l2_arr.append(coef)
    mse_arr.append(mae_score)

# Save results in df
resl = pd.DataFrame(np.array(mse_arr), columns=['mse_score'])\
             .merge(pd.DataFrame(np.array(l2_arr), columns=['L2 coefs']), right_index=True, left_index=True)
resl

,mse_score,L2 coefs
0,440.194630,0.0
1,2723.100877,0.1
2,3497.215908,0.2
3,1574.678353,0.3
4,1729.045533,0.4
5,4846.240371,0.5
6,5463.899679,0.6
7,2209.582291,0.7
8,1714.712690,0.8
9,1720.653458,0.9


In [88]:
ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(rfm_train_test['frequency_cal'],
        rfm_train_test['monetary_value_cal'])

<lifetimes.GammaGammaFitter: fitted with 1956 subjects, p: 2.33, q: 3.34, v: 413.37>

##### **Prediction**
##### We want to:
##### &emsp;1. Use the BG/NBD model to predict the expected number of transactions.
##### &emsp;2. Use the Gamma-Gamma model to predict the average order value.
##### &emsp;3. Multiply the average order value by the expected number of transactions to get the customer's revenue.

In [89]:
#Predict the expected number of transactions in the next 89 days
predicted_bgf = bgf.predict(89,
                        rfm_train_test['frequency_cal'], 
                        rfm_train_test['recency_cal'], 
                        rfm_train_test['T_cal'])
trans_pred = predicted_bgf.fillna(0)

#Predict the average order value
monetary_pred = ggf.conditional_expected_average_profit(rfm_train_test['frequency_cal'],
                                         rfm_train_test['monetary_value_cal'])

sales_preds = trans_pred * monetary_pred

##### **Model Evaluation**

In [90]:
actual = rfm_train_test['monetary_value_holdout'] *  rfm_train_test['frequency_holdout']

In [91]:
def evaluate_model(actual, preds):
    print(f"Total Sales Actual: {np.round(actual.sum())}")
    print(f"Total Sales Predicted: {np.round(preds.sum())}")
    print(f'Mean absolute error: {mean_absolute_error(actual, preds)}')
    #merged = pd.merge(pd.DataFrame(bgf_preds), pd.DataFrame(rfm_train_test['frequency_holdout']), on=['id'])
    fig = px.scatter(x=np.array(preds), y=np.array(actual), title='Predicted vs Actual')
    fig.update_layout(
        xaxis_title='Predicted',
        yaxis_title='Actual'
    )
    fig.show()

In [92]:
evaluate_model(actual, sales_preds)

Total Sales Actual: 1885817.0
Total Sales Predicted: 1474134.0
Mean absolute error: 601.5708046888286


# **Talk about comparison with LightGBM**

In [ ]:
#### **ML Approach**